In [1]:
import wbdata
import pandas as pd
import requests
from datetime import datetime
# Retrieve indicators
result = wbdata.get_indicators()

# Convert the result to a DataFrame
df = pd.DataFrame(result)

# Display the first few rows of the DataFrame
print(df.head())

# Filter indicators related to debt
debt_indicators = df[df['name'].str.contains('debt', case=False, na=False)]

# Save the DataFrame to a CSV file
debt_indicators.to_csv('debt_indicators.csv', index=False)

print("CSV file 'debt_indicators.csv' has been created.")


                     id                                     name unit  \
0    1.0.HCount.1.90usd          Poverty Headcount ($1.90 a day)        
1     1.0.HCount.2.5usd          Poverty Headcount ($2.50 a day)        
2  1.0.HCount.Mid10to50    Middle Class ($10-50 a day) Headcount        
3       1.0.HCount.Ofcl  Official Moderate Poverty Rate-National        
4   1.0.HCount.Poor4uds             Poverty Headcount ($4 a day)        

                                    source  \
0  {'id': '37', 'value': 'LAC Equity Lab'}   
1  {'id': '37', 'value': 'LAC Equity Lab'}   
2  {'id': '37', 'value': 'LAC Equity Lab'}   
3  {'id': '37', 'value': 'LAC Equity Lab'}   
4  {'id': '37', 'value': 'LAC Equity Lab'}   

                                          sourceNote  \
0  The poverty headcount index measures the propo...   
1  The poverty headcount index measures the propo...   
2  The poverty headcount index measures the propo...   
3  The poverty headcount index measures the propo...   
4  T

In [2]:
debt_indicators_df = pd.read_csv('debt_indicators.csv')

# Inspect the DataFrame to understand its structure
print(debt_indicators_df.head())

               id                                               name  unit  \
0  BN.DSR.UNPD.CD           Debt service not paid (BoP, current US$)   NaN   
1  BN.FAC.ARAC.CD  Debt Service not paid: Arrears Accumulation (B...   NaN   
2       CRPFM-7.2                Climate-related debt and guarantees   NaN   
3  DB.DOD.DLXF.CD             Central Government External Debt (US$)   NaN   
4  DE.DOD.DLXF.CD     Non-financial Pub. Enterprises Ext. Debt (US$)   NaN   

                                              source  \
0  {'id': '11', 'value': 'Africa Development Indi...   
1  {'id': '11', 'value': 'Africa Development Indi...   
2                {'id': '91', 'value': 'PEFA_CRPFM'}   
3     {'id': '57', 'value': 'WDI Database Archives'}   
4     {'id': '57', 'value': 'WDI Database Archives'}   

                                          sourceNote  \
0         Debt service not paid (BoP, current US$).    
1  In the standard presentation of the balance of...   
2                         

In [3]:
# Extract IDs from the DataFrame
indicator_ids = debt_indicators_df['id'].tolist()

In [5]:
import pandas as pd
import requests

# Load the CSV file with indicator IDs
debt_indicators_df = pd.read_csv('debt_indicators.csv')

def fetch_indicator_data(indicator_id):
    url = f"http://api.worldbank.org/v2/indicator/{indicator_id}?format=json&per_page=50000"
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()
    # Print the first few entries to inspect the structure
    print(data)
    return data

def extract_annual_data(data, indicator_id):
    annual_data = {}
    if data and isinstance(data, list) and len(data) > 1:
        indicator_data = data[1]  # The actual data is usually in the second item
        for entry in indicator_data:
            # Check for available keys
            year = entry.get('date')
            amount = entry.get('value')
            if year and amount is not None:
                if year not in annual_data:
                    annual_data[year] = {}
                annual_data[year][indicator_id] = amount
    return annual_data

def organize_data(indicator_ids):
    all_data = {}
    
    for indicator_id in indicator_ids:
        try:
            data = fetch_indicator_data(indicator_id)
            annual_data = extract_annual_data(data, indicator_id)
            for year, values in annual_data.items():
                if year not in all_data:
                    all_data[year] = {}
                all_data[year][indicator_id] = values.get(indicator_id, None)
        except requests.HTTPError as e:
            print(f"Failed to fetch data for indicator ID {indicator_id}: {e}")
    
    # Convert to DataFrame
    df = pd.DataFrame.from_dict(all_data, orient='index').sort_index()
    df.reset_index(inplace=True)
    df.rename(columns={'index': 'Year'}, inplace=True)
    return df

# Extract indicator IDs
indicator_ids = debt_indicators_df['id'].tolist()

# Organize data
data_df = organize_data(indicator_ids)

# Save to CSV
#data_df.to_csv('debt_indicators_data.csv', index=False)

#print("Data has been successfully saved to 'debt_indicators_data.csv'.")


[{'page': 1, 'pages': 1, 'per_page': '50000', 'total': 1}, [{'id': 'BN.DSR.UNPD.CD', 'name': 'Debt service not paid (BoP, current US$)', 'unit': '', 'source': {'id': '11', 'value': 'Africa Development Indicators'}, 'sourceNote': 'Debt service not paid (BoP, current US$). ', 'sourceOrganization': 'World Bank country economists.', 'topics': [{}]}]]
[{'page': 1, 'pages': 1, 'per_page': '50000', 'total': 1}, [{'id': 'BN.FAC.ARAC.CD', 'name': 'Debt Service not paid: Arrears Accumulation (BoP, current US$)', 'unit': '', 'source': {'id': '11', 'value': 'Africa Development Indicators'}, 'sourceNote': 'In the standard presentation of the balance of payments, arrears of interest and amortization--amounts that are past due and unpaid--are recorded as if the amounts had been paid on schedule, and an offsetting entry is made to reflect the associated new, short-term commitments.  Data are in current US dollars. ', 'sourceOrganization': 'World Bank country economists.', 'topics': [{}]}]]
[{'page': 1